In [ ]:
import numpy as np
import pandas as pd

from glob import glob

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
# point to directory with csv files data
matching_files = glob('../../data/matching_phase1vsphase2/*.csv')
matching_files

In [ ]:
matching_opr = pd.concat(pd.read_csv(file, index_col=False) for file in matching_files)
matching_opr.sample(10)

Total number of operators per country

In [ ]:
df = matching_opr.groupby('COUNTRY_CODE')[['name_phase1', 'placeId']].agg(['count'])
df.columns = ['total_nr_oprs', 'nr_matches']
df['percent_matched'] = 100 * df.nr_matches / df.total_nr_oprs
df

Average of all similarities found per country

In [ ]:
matching_opr.dropna(subset=['SIMILARITY']).groupby('COUNTRY_CODE').mean()

Average of top similarities per operator per country

In [ ]:
(matching_opr
 .dropna(subset=['SIMILARITY'])
 .sort_values('SIMILARITY', ascending=False)
 .drop_duplicates('name_phase1', keep='first')
 .groupby('COUNTRY_CODE')
 .mean())

In [ ]:
def get_batch_matches(matching_output, country_code, nr_operators=100, only_matches=True, threshold=.7, only_highest=False, columns=None, ascending=False):

    df = matching_output.loc[(matching_output.COUNTRY_CODE == country_code) & (matching_output.SIMILARITY >= threshold)]
    if only_matches:
        names = df.dropna(axis=0, subset=['placeId']).name_phase1.unique()
    else:
        names = df.name_phase1.unique()
    
    names = np.random.choice(names, size=nr_operators, replace=False)
        
    columns = columns or ['name_phase1', 'name_phase2', 'SIMILARITY']
    df = df.loc[df.name_phase1.isin(names), columns]
    if only_highest:
        df = (df
              .sort_values('SIMILARITY', ascending=False)
              .drop_duplicates('name_phase1', keep='first'))
    return df.sort_values(by=['name_phase1', 'SIMILARITY'], ascending=ascending)

In [ ]:
batch_opr = get_batch_matches(matching_opr, 'NZ', nr_operators=10, only_highest=True)
batch_opr.describe()

In [ ]:
batch_opr